In [1]:
pip install pandas requests beautifulsoup4 tqdm


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.13/bin/python3.13 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

# ✅ 청원 요지 크롤링 함수
def get_petition_summary(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            summary_element = soup.select_one("div.textType02#summaryContentDiv")
            if summary_element:
                return summary_element.text.strip()
            return None
        else:
            print(f"❌ 요청 실패: {url}")
            return None
    except Exception as e:
        print(f"❌ 크롤링 오류: {e}")
        return None

# ✅ OpenAPI로 청원 데이터 가져오기
def fetch_petition_data(api_key, start_eraco=13, end_eraco=22):
    base_url = "https://open.assembly.go.kr/portal/openapi/PTTRCP"
    all_data = []

    for d in range(start_eraco, end_eraco + 1):
        eraco = f"제{d}대"
        p_index = 1
        print(f"📥 수집 중: {eraco}")
        while True:
            params = {
                "KEY": api_key,
                "Type": "json",
                "pIndex": p_index,
                "pSize": 100,
                "ERACO": eraco
            }
            try:
                response = requests.get(base_url, params=params)
                if response.status_code != 200:
                    print(f"❌ 요청 실패 (ERACO={eraco}, pIndex={p_index})")
                    break

                # ✅ 응답 구조 처리
                response_json = response.json()
                pttrcp_data = response_json.get("PTTRCP", [])
                result = []
                for section in pttrcp_data:
                    if isinstance(section, dict) and "row" in section:
                        result = section["row"]
                        break

                if not result:
                    break

                all_data.extend(result)
                p_index += 1
                time.sleep(0.2)

            except Exception as e:
                print(f"❌ 오류 발생: {e}")
                break

    return all_data




# ✅ 메인 실행
def main():
    api_key = "fd2ed9146ff64b59b775efb194846209"  # ← 발급받은 인증키로 변경하세요

    # 1. API에서 데이터 수집
    petition_data = fetch_petition_data(api_key)

    # 2. 데이터프레임 변환
    df = pd.DataFrame(petition_data)
    print("📌 컬럼 목록:", df.columns.tolist())  # ← 이 줄 추가


    # 3. 'LINK_URL' → '링크URL'로 별칭 맞추기
    if "LINK_URL" in df.columns:
        df["링크URL"] = df["LINK_URL"]  # ✅ 기존 코드 호환용 컬럼명 설정

        # 4. '링크URL'을 이용해 청원 요지 수집
        summary_list = []
        print("🔍 청원 요지 크롤링 중...")
        for url in tqdm(df["링크URL"]):
            summary = get_petition_summary(url)
            summary_list.append(summary if summary else "")
        df["청원요지"] = summary_list
    else:
        print("⚠️ 'LINK_URL' 컬럼이 존재하지 않습니다.")

    # 5. CSV 저장
    df.to_csv("병합된_청원_데이터.csv", index=False, encoding="utf-8-sig")
    print("✅ 병합된 CSV 저장 완료: 병합된_청원_데이터.csv")

# ✅ 실행
if __name__ == "__main__":
    main()

📥 수집 중: 제13대
📥 수집 중: 제14대
📥 수집 중: 제15대
📥 수집 중: 제16대
📥 수집 중: 제17대
📥 수집 중: 제18대
📥 수집 중: 제19대
📥 수집 중: 제20대
📥 수집 중: 제21대
📥 수집 중: 제22대
📌 컬럼 목록: ['CITZN_AGM_CNT', 'ERACO', 'LINK_URL', 'INTD_ASBLM_NM', 'RCP_DT', 'PTT_ID', 'PTT_NM', 'PTT_NO', 'PTTR_NM', 'PTT_KIND']
🔍 청원 요지 크롤링 중...


 11%|█████████▉                                                                                 | 424/3906 [01:37<13:16,  4.37it/s]